In [9]:
import pandas as pd
import numpy as np
import json
from datetime import datetime

# Cargar archivo JSON
# Reemplaza 'archivo.json' con la ruta a tu archivo
df = pd.read_json('datos_sucios_mongo.json')

# Mostrar primeras filas
df.head()


,nombre,correo,edad,telefono,ciudad,fecha_registro
0,Dafne Alicia Segovia Pacheco,jllorens@larrea-figueroa.org,veinticinco,None,PUERTO MONTT,2024-11-05
1,Samu Marcos Antón,cabreracandelaria@gmail.com,19,,Concepción,
2,Felisa de Llamas,cabreroelba@muro.es,None,123456,Puerto Montt,None
3,Fito Fortuny Iniesta,plinioropero@solano.es,None,123456,la serena,None
4,Bartolomé Viña,mtorrijos@diez.com,,None,Concepción,None


In [10]:
# % de valores no nulos por columna
def evaluar_completitud(df):
    completitud = df.notnull().mean().round(3) * 100
    return completitud.to_frame(name='Porcentaje de completitud (%)')


In [11]:
# Proporción de duplicados
def evaluar_unicidad(df):
    total = len(df)
    duplicados = df.duplicated().sum()
    unicidad = 100 * (1 - duplicados / total)
    return {'Duplicados': duplicados, 'Porcentaje Unicidad': round(unicidad, 2)}


In [12]:
# Detectar datos fuera de rango
def evaluar_validez(df, reglas):
    resultados = {}
    
    for columna, regla in reglas.items():
        if columna not in df.columns:
            resultados[f'{columna}_no_existe'] = 'Columna no encontrada'
            continue

        tipo_esperado = regla.get('type')
        if tipo_esperado:
            mask_tipo_invalido = ~df[columna].apply(lambda x: isinstance(x, tipo_esperado))
            resultados[f'{columna}_tipo_invalido'] = mask_tipo_invalido.sum()
        
        if 'range' in regla:
            rango_min, rango_max = regla['range']
            mask_valida = df[columna].apply(lambda x: isinstance(x, tipo_esperado))
            valores_validos = df.loc[mask_valida, columna]
            fuera_rango = valores_validos[(valores_validos < rango_min) | (valores_validos > rango_max)]
            resultados[f'{columna}_fuera_rango'] = len(fuera_rango)

    return resultados


In [13]:
def evaluar_consistencia(df):
    resultados = {}

    # Regla 1: Si no hay teléfono, tampoco debería haber correo (ejemplo artificial)
    inconsistencia_tel_correo = df[(df['telefono'].isnull()) & (df['correo'].notnull())]
    resultados['telefono_nulo_correo_presente'] = len(inconsistencia_tel_correo)

    return resultados


In [14]:
#Comparar columnas relacionadas
def evaluar_actualidad(df, col_fecha):
    if col_fecha not in df.columns:
        return {f'{col_fecha}': 'Columna no encontrada'}
    
    hoy = pd.to_datetime(datetime.today())
    fechas = pd.to_datetime(df[col_fecha], errors='coerce')
    antiguedad_dias = (hoy - fechas).dt.days
    return {
        'Media antigüedad (días)': antiguedad_dias.mean(),
        'Máxima antigüedad (días)': antiguedad_dias.max()
    }


In [15]:
# Reglas de validez ajustadas a tus columnas
reglas_validez = {
    'edad': {'type': int, 'range': (0, 120)},
    'telefono': {'type': str},
    'correo': {'type': str},
}

# Ejecutar funciones
completitud = evaluar_completitud(df)
unicidad = evaluar_unicidad(df)
validez = evaluar_validez(df, reglas_validez)
consistencia = evaluar_consistencia(df)
actualidad = evaluar_actualidad(df, 'fecha_registro')

In [16]:
#Reporte de problemas encontrados

print(" COMpletitud:")
display(completitud)

print("\n Unicidad:")
print(unicidad)

print("\n Validez:")
for k, v in validez.items():
    print(f"{k}: {v} registros problemáticos")

print("\n Consistencia:")
for k, v in consistencia.items():
    print(f"{k}: {v} registros inconsistentes")

print("\n Actualidad:")
for k, v in actualidad.items():
    print(f"{k}: {round(v, 2) if isinstance(v, (int, float)) else v}")


 COMpletitud:


,Porcentaje de completitud (%)
nombre,100.0
correo,100.0
edad,75.4
telefono,76.4
ciudad,100.0
fecha_registro,69.2



 Unicidad:
{'Duplicados': 0, 'Porcentaje Unicidad': 100.0}

 Validez:
edad_tipo_invalido: 375 registros problemáticos
edad_fuera_rango: 0 registros problemáticos
telefono_tipo_invalido: 118 registros problemáticos
correo_tipo_invalido: 0 registros problemáticos

 Consistencia:
telefono_nulo_correo_presente: 118 registros inconsistentes

 Actualidad:
Media antigüedad (días): 377.87
Máxima antigüedad (días): 723.0


#### Criterios de evaluación de calidad de datos

- **Completitud**: porcentaje de valores no nulos por columna.
- **Unicidad**: porcentaje de filas no duplicadas.
- **Validez**: tipo de dato esperado y rangos aceptables.
- **Consistencia**: relaciones lógicas entre columnas (e.g., fecha inicio < fecha fin).
- **Actualidad**: número de días desde la última actualización del dato.

Cada métrica permite identificar registros problemáticos y cuantificar su impacto.
